In [1]:
import torch
from chadhmm.hmm import MultinomialHMM, GaussianHMM, GaussianMixtureHMM, PoissonHMM

In [2]:
hmm = MultinomialHMM(n_states=6,
                     n_features=4,
                     n_trials=2,
                     transitions='ergodic')
train_seq = torch.randint(0,hmm.n_features,(1000,))
one_hot = hmm.n_trials * torch.nn.functional.one_hot(train_seq,4)

In [3]:
hmm.fit(X=one_hot,max_iter=10,n_init=1,verbose=False)
print(f'pi {hmm._params.pi.exp()}\n' +
      f'A  {hmm._params.A.exp()}\n' +
      f'B  {hmm._params.B.exp()}')

pi tensor([0.0000, 0.0000, 0.0014, 0.0000, 0.9986, 0.0000], dtype=torch.float64)
A  tensor([[0.2165, 0.0719, 0.0845, 0.0407, 0.0017, 0.5847],
        [0.0670, 0.1055, 0.0900, 0.4600, 0.0923, 0.1852],
        [0.0191, 0.0797, 0.2333, 0.4738, 0.0019, 0.1921],
        [0.0251, 0.1017, 0.2553, 0.4217, 0.0064, 0.1898],
        [0.0297, 0.0086, 0.0649, 0.0564, 0.1398, 0.7006],
        [0.0358, 0.0537, 0.3344, 0.4696, 0.0039, 0.1026]], dtype=torch.float64)
B  tensor([[ 1.4284e-29,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  0.0000e+00,  2.9656e-12,  0.0000e+00],
        [2.1077e-270,  1.0000e+00,  0.0000e+00, 3.3356e-281],
        [ 4.1480e-01,  0.0000e+00,  0.0000e+00,  5.8520e-01],
        [1.2015e-108,  1.0000e+00,  0.0000e+00,  5.2371e-10],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00, 4.7455e-250]],
       dtype=torch.float64)


In [4]:
hmm.ic(one_hot,lengths=[600,400])

tensor([2077.4262, 1422.8533], dtype=torch.float64)

In [5]:
hmm.score(one_hot,lengths=[600,400])

tensor([-985.7131, -658.4267], dtype=torch.float64)

In [11]:
hmm.predict(one_hot)

IndexError: index 6 is out of bounds for dimension 0 with size 6

In [2]:
gmm = GaussianHMM(n_states=2,n_features=2)
train_seq = torch.rand((1000,2), dtype=torch.float64)
gmm.fit(X=train_seq,max_iter=5, n_init=1, verbose=True)

Run 1 | Initialization | Score: -2175.26
Run 1 | Iteration: 1 | Score: -914.80 | Delta: 1260.47 | Converged = False
Run 1 | Iteration: 2 | Score: -324.68 | Delta: 590.12 | Converged = False
Run 1 | Iteration: 3 | Score: -324.66 | Delta: 0.01 | Converged = False
Run 1 | Iteration: 4 | Score: -324.65 | Delta: 0.02 | Converged = False
Run 1 | Iteration: 5 | Score: -324.63 | Delta: 0.02 | Converged = False


GaussianHMM(
  (_params): ParameterDict(
      (covs): Parameter containing: [torch.DoubleTensor of size 2x2x2]
      (means): Parameter containing: [torch.DoubleTensor of size 2x2]
      (A): Parameter containing: [torch.DoubleTensor of size 2x2]
      (pi): Parameter containing: [torch.DoubleTensor of size 2]
  )
)

In [4]:
phmm = PoissonHMM(n_states=2,n_features=2)
train_seq = torch.randint(1,100,(100,2))
phmm.fit(X=train_seq,max_iter=15,lengths=[50,50],n_init=1,verbose=False)

PoissonHMM(
  (_params): ParameterDict(
      (rates): Parameter containing: [torch.DoubleTensor of size 2x2]
      (A): Parameter containing: [torch.DoubleTensor of size 2x2]
      (pi): Parameter containing: [torch.DoubleTensor of size 2]
  )
)

In [12]:
gmmm = GaussianMixtureHMM(n_states=4,
                         n_components=3,
                         n_features=3)
train_seq = torch.rand((100,3),dtype=torch.float64)
#gmmm.fit(X=train_seq,max_iter=15, n_init=1, lengths=[50,50], verbose=False)

In [13]:
gmmm.pdf.component_distribution.log_prob(train_seq)

ValueError: Value is not broadcastable with batch_shape+event_shape: torch.Size([100, 3]) vs torch.Size([4, 3, 3]).

In [8]:
gmmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

UnboundLocalError: local variable 'seq' referenced before assignment

In [25]:
emiss = gmmm.pdf.component_distribution
x = train_seq

pdf_shape =  emiss.batch_shape + emiss.event_shape
b_size = torch.Size([torch.atleast_2d(x).size(0)]) + pdf_shape
x_batched = x.unsqueeze(0).expand(b_size)
emiss.log_prob(x_batched).squeeze()

RuntimeError: The expanded size of the tensor (3) must match the existing size (100) at non-singleton dimension 2.  Target sizes: [100, 4, 3, 3].  Tensor sizes: [1, 100, 3]

In [ ]:
import torch
from chadhmm.hsmm import CategoricalHSMM, GaussianHSMM, GaussianMixtureHSMM, PoissonHSMM

In [ ]:
hsmm = CategoricalHSMM(n_states=5,
                       n_features=4,
                       max_duration=4)
train_seq = torch.randint(0, 4, (100,))
hsmm.fit(X=train_seq,max_iter=15, n_init=1)

In [ ]:
print(f'pi {hsmm.params.pi.exp().sum()}\n'+
      f'A  {hsmm.params.A.exp().sum(1)}\n' +
      f'D  {hsmm.params.D.exp().sum(1)}\n' +
      f'B  {hsmm.params.B.exp().sum(1)}')

In [ ]:
ghsmm = GaussianHSMM(n_states=5,
                  n_features=3,
                  max_duration=4)
train_seq = torch.rand((100,3), dtype=torch.float64)
ghsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
gmhsmm = GaussianMixtureHSMM(n_states=4,
                         n_components=3,
                         max_duration=4,
                         n_features=3)
train_seq = torch.rand((100,3),dtype=torch.float64)
gmhsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical, Multinomial

Multinomial(2,probs=torch.Tensor([0.2,0.8]))

In [ ]:
m = Multinomial(1,logits=hmm.pdf.logits)
m.log_prob(torch.Tensor([0,0,0,1])).exp()

In [ ]:
hmm.pdf.log_prob(train_seq[4])